In [1]:
from opentronsjson import plates,commands
import opentronsjson
import datetime
import uuid
import json
import itertools
from operator import itemgetter
with open('./../protocol-schema.json') as f:
    data = json.load(f)
from jsonschema import validate


with open('/home/koeng/Downloads/response_1554406909932.json') as f:
    test_plate = json.load(f)

opentrons = {'uuid': '43346aba-41e0-483d-ad43-d46a85a6ead5',
             'right_300': '9c3b3205-4fe3-4bf1-af3f-8cf9ab95d918',
             'left_10': '82e95a7a-f335-4178-a7b6-4f38954e497a'
            }

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import string
class Plate():
    def __init__(self, height:int=8, length:int=12,locations:list=[]) -> None:
        positions = []
        for letter in list(string.ascii_uppercase[0:height]):
            for number in range(length):
                positions.append((letter, number+1))
        self.height = height
        self.length = length
        if locations==[]:
            self.locations = [x[0]+str(x[1]) for x in positions]
        else:
            self.locations = locations

    def rows(self)-> list:
        #tuples good strings bad
        return [list(g) for k, g in groupby(self.locations, key=lambda x: x[0])]
    def columns(self) -> list:
        # why not the same above, sed 's/x[0]/x[1:]'
        return [[x for x in self.locations if x[1:] == str(num+1)] for num in range(self.length)]
    def reverse_columns(self)-> list:
        return [sorted(x, reverse=True) for x in self.columns()]
    def flat_columns(self) -> list:
        # use a classmethod constructor
        # expose only public locations, wells (dict of dict of well), rows, columns
        return [item for sublist in self.columns() for item in sublist]


    def subcolumns(self)-> list:
        if self.height%2 != 0:
            raise StandardError('Not compatible with multichannel')
        spaces = int(self.height/8)
        subcolumns=[]
        for column in self.columns():
            for space in range(spaces):
                count = space
                subcolumn=[]
                while count < len(column):
                    subcolumn.append(column[count])
                    count+=spaces
                subcolumns.append(subcolumn)
        return subcolumns
    
    def use_wells(self,tips:list)-> str:
        self.locations = [v for i, v in enumerate(self.locations) if v not in tips]
        return tips

    def use_number(self,num_wells:int=8,columns=None) -> str:
        if columns == None:
            columns=self.reverse_columns()
        for column in columns:
            if len(column) >= num_wells:
                return self.use_wells(column[0:num_wells])
        raise ValueError("No more tips.")

    def derived_subcolumns(self,loc:list):
        return [[well if well in loc else None for well in col] for col in self.subcolumns()]

        
def data_transfer_comprehension(data_transfer, old_plate=Plate(),new_plate=Plate()):
    output=[]
    for transfer in data_transfer:
        original_index = old_plate.flat_columns().index(transfer[1])
        new_index = new_plate.flat_columns().index(transfer[2])
        for i in range(transfer[0]):
            old_well = old_plate.flat_columns()[i+original_index]
            new_well = new_plate.flat_columns()[i+new_index]
            output.append((old_well,new_well))
    return output
            

def efficient_transfer(derived:list,full=Plate().subcolumns()):
    derived = Plate(locations=derived).subcolumns()
    transfers=[]
    well = None
    for index,subcolumn in enumerate(derived):
        tip_count = 1
        loc_num=0
        wells=[]
        counter = 0
        for well_index,well in enumerate(subcolumn):

            # Check if this is a new well
            wells.append(well)

            # Check that this is not the last location we can pull from 
            if not well_index+2 > len(subcolumn):
                loc_num = full[index].index(subcolumn[well_index])-well_index-counter



            # If last location, add tips and location, reset new well
            if loc_num != 0:
                if loc_num != 50:
                    transfers.append((len(wells[:-1]),wells[:-1]))
                    wells=[wells[-1]]
                    tip_count=0
                    counter=loc_num
                else:
                    transfers.append((len(wells),wells))
                    wells=[]
                    tip_count=0
                    counter=loc_num
            
            loc_num=50
            tip_count+=1
    return transfers

In [3]:
def transfer(volume,from_wells,tipbox=Plate(),from_plate_layout=Plate(),to_plate_layout=Plate()):
    def efficient_tip(derived,full):
        transfers=[]
        well = None
        for index,subcolumn in enumerate(derived):
            tip_count = 1
            loc_num=0
            wells=[]
            for well_index,well in enumerate(subcolumn):

                # Check if this is a new well
                wells.append(well)

                # Check that this is not the last location we can pull from 
                if not well_index+2 > len(subcolumn):
                    loc_num = full[index].index(subcolumn[well_index+1])-well_index

                # If last location, add tips and location, reset new well
                if loc_num != 1:
                    transfers.append((tip_count,wells))
                    wells=[]
                    tip_count=0

                loc_num=0
                tip_count+=1
        return transfers
    
    transfers = efficient_tip(Plate(locations=from_wells).subcolumns(),from_plate_layout.subcolumns())
    to = []
    for step in transfers:
        tips = tipbox.use_number(step[0])
        to+=to_plate_layout.use_number(step[0],to_plate_layout.columns())
    transfer_from = [item for sublist in [step[1] for step in transfers] for item in sublist]
    
    individual_transfers = [(volume,) + transfer for transfer in list(zip(transfer_from,to))]
    return transfers,individual_transfers

In [4]:
class Protocol():
    def __init__(self,robot:dict,labware:dict,procedure:dict,protocol_schema:str='1.0.0',metadata:dict={},defaults:dict={},designer_application:dict={},pipettes:dict={}):
        # Metadata
        self.protocol_schema = protocol_schema
        self.metadata = metadata
        
        # Defaults
        if defaults == {}:
            self.defaults = opentronsjson.default_values
        else:
            self.defaults = defaults
        
        #Designer application
        if designer_application == {}:
            self.designer_application = opentronsjson.designer_application
        else:
            self.designer_application = designer_application
            
        # Pipettes 
        if pipettes == {}:
            self.pipettes = {
                robot['left_10']: opentronsjson.pipette("left", "p10_multi", "p10_multi_v1.3"),
                robot['right_300']: opentronsjson.pipette("right", "p300_multi", "p300_multi_v1.3")
            }
        else:
            self.pipettes = pipettes
        
        self.labware = labware
        self.procedure=procedure
            
    def toJSON(self):
         return {
             "protocol-schema": self.protocol_schema,
             "metadata": self.metadata,
             "default-values": self.defaults,
             "designer-application": self.designer_application,
             "robot": opentronsjson.ot2_standard,
             "pipettes": self.pipettes,
             "labware": self.labware,
             "procedure": self.procedure}
        
def plate_merge(robot,plate,breadcrumb=None,strain='E.coli Top10',transformant_volume=15,media='lb',transformation_volume=2):
    # Create new plate
    new_plate = {"plate_form": "96pcr",
                 "plate_name": "{}-transformation".format(plate['plate_name']),
                 "plate_type": "transformation",
                 "status": "planned",
                 "breadcrumb": breadcrumb,
                 "uuid": str(uuid.uuid4())}

    # Metadata
    metadata = {
            "protocolName": "Plate merge to create {}".format(plate['uuid']),
            "author": "Keoni Gandall",
            "datetime": "{}".format(str(datetime.datetime.now()))
          }

    # Addresses from
    addresses = []
    for well in plate['wells']:
        addresses.append(well['address'])

    # Tipbox
    tipbox = Plate()
    new = Plate()

    # Labware
    pipette = robot['left_10']
    tips = '11_tip_box'

    labware = {
        tips: opentronsjson.labware('11',"opentrons-tiprack-10ul",name="11_tip_box"),
        'trash_yo': opentronsjson.labware('12',"fixed-trash",name="Trash yo"),
        new_plate['uuid']: opentronsjson.labware('5', '96-PCR-flat', name=new_plate['plate_name']),
        plate['uuid']: opentronsjson.labware('4', '96-PCR-flat', name=plate['plate_name'])
    }
    
    # Create new procedure
    procedure = []
    data_transfer = []
    high_level_transfers = efficient_transfer(addresses)
    volume = transformation_volume
    for transfer in high_level_transfers:
        procedure.append(commands.pick_up_tip(pipette,tips,tipbox.use_number(transfer[0])[-1]))
        transfer_from = transfer[1][0]
        procedure.append(commands.aspirate(pipette,volume,plate['uuid'],transfer_from))
        transfer_to = new.use_number(transfer[0],columns=new.columns())[0]
        procedure.append(commands.dispense(pipette,volume,new_plate['uuid'],transfer_to))
        procedure.append(commands.drop_tip(pipette,'trash_yo','A1'))
        data_transfer.append((transfer[0],transfer_from,transfer_to))
    
    protocol = {'uuid': str(uuid.uuid4()),
               'description': metadata['protocolName'],
               'protocol': Protocol(robot,labware,[{'subprocedure': procedure}],metadata=metadata).toJSON(),
               'status': 'planned'}
    
    transfers = data_transfer_comprehension(data_transfer)
    new_wells=[]
    for transfer in transfers:
        old_well = {}
        for well in plate['wells']:
            if well['address'] == transfer[0]:
                old_well = well
        new_well = {'address': transfer[1],
                    'media': media,
                    'organism': strain,
                    'plate_uuid': new_plate['uuid'],
                    'quantity': None,
                    'samples': [sample['uuid'] for sample in old_well['samples']],
                    'volume': transformant_volume+transformation_volume,
                   'well_type': 'transformation'}
        new_wells.append(new_well)
    
    output = {'Plate': [new_plate],
             'Well': new_wells,
             'Protocol': [protocol]}
    return output
    
        

result = transformation(opentrons,test_plate)#['protocol']
#validate(result,schema=data)

NameError: name 'transformation' is not defined

In [5]:
import requests
r = requests.get('https://api.freegenes.org/plates/recurse/dcf981a7-683c-4bad-acfe-8ed35217ec36').json()

In [28]:
def plates_join_parts(parts:list,plates:list,status=None): 
    """Can merge maximum 6 plates into 1. Parts are expressed as UUIDs. Plates are plate recursions """
    used_parts = []
    plate_dict = {}
    # for plate in plates
    #     for part in parts:
    #          if part in plate"
    #              used
    for plate in plates:
        target_wells = []
        for well in plate['wells']:
            for sample in well['samples']:
                part_uuid = sample['part']['uuid']
                if part_uuid in parts and part_uuid not in used_parts:
                    #if sample['status'] == 'Confirmed':
                    if sample['status'] == None:
                        target_wells.append(well['address'])
                        used_parts.append(part_uuid)
        plate_dict[plate['uuid']] = target_wells
    return plate_dict

def group_locs(locs,plate=Plate()):
    # this is good with iter
    # sad without
    transfers = []
    for col in plate.derived_subcolumns(locs):
        trans = []
        for i,well in enumerate(col):
            if well != None:
                trans.append(well)

            if i+1 == len(col):
                if trans != []: transfers.append(trans)
            else:
                if col[i+1] == None:
                    if trans != []: transfers.append(trans)
                    trans = []
    return transfers

def high_level_transfer(parts:list,plates:list,status=None):
    plate_dict = plates_join_parts(parts,plates,status)
    return {k: group_locs(v) for k,v in plate_dict.items()}

def plate_merge(name:str,plate_uuid:str,plate_num:int,plate_dict:dict,robot:dict,breadcrumb:None):
    new_plate = {"plate_form": "standard96",
                 "plate_name": "{}-distro-{}".format(name,str(plate_num)),
                 "plate_type": "distribution_glycerol_stock",
                 "status": "planned",
                 "breadcrumb": breadcrumb,
                 "uuid": plate_uuid}
    # Metadata
    metadata = {
            "protocolName": "Plate merge to create {}".format(new_plate['plate_name']),
            "author": "Keoni Gandall",
            "datetime": "{}".format(str(datetime.datetime.now()))
          }


    # Tipbox
    tipbox = Plate()
    new = Plate()

    # Labware
    pipette = str(uuid.uuid4())
    tips = '11_tip_box'

    labware = {
        tips: opentronsjson.labware('11',"opentrons-tiprack-300ul",name="11_tip_box"),
        'trash_yo': opentronsjson.labware('12',"fixed-trash",name="Trash yo"),
        new_plate['uuid']: opentronsjson.labware('10', '96-deep-well', name=new_plate['plate_name']),
    }
    for i,k in enumerate(list(set([d['from'] for d in plate_dict]))):
        labware[k] = opentronsjson.labware(str(i+4), '96-flat', name=k)
    

    procedure = []
    vol=7
    
    to_wells = []
    from_wells = []
    for d in plate_dict:
        procedure.append(commands.pick_up_tip(pipette,tips,tipbox.use_number(len(d['trans']))[-1]))
        procedure.append(commands.aspirate(pipette,vol,d['from'],d['trans'][0]))
        from_wells.append((d['from'],d['trans']))
        to_wells.append((d['to'],new.use_number(len(d['trans']),columns=new.subcolumns())))
        procedure.append(commands.dispense(pipette,vol,d['to'], to_wells[-1][1][-1]))
        procedure.append(commands.drop_tip(pipette,'trash_yo','A1'))

    for from_well,to_well in zip(from_wells,to_wells):
        print(from_wells, to_wells)
        for f_w,t_w in zip(from_well[1],to_well[1]):
            #print(f_w)
            #print(t_w)
            pass
        
    protocol = Protocol({},labware,[{'subprocedure': procedure}],metadata=metadata,pipettes={pipette:opentronsjson.pipette("left", "p50_multi", "p50_multi_v1.3")}).toJSON()
    
    
    return protocol

def compile_parts(collection_name:str,parts:list,plates:list,status='Confirmed'):
    groups = high_level_transfer(parts,plates,status)
    num_parts = sum([sum([len(tran) for tran in v])for k,v in groups.items()])
    plate_num = int(num_parts/96) + 1

    # Gen protocol
    new_plate = str(uuid.uuid4())
    protocol = []
    count = 0
    for k,v in groups.items():
        for trans in v:
            if count+len(trans)>96: 
                print('new')
                new_plate = str(uuid.uuid4())
                count=0
            else:
                protocol.append({'from': k, 'to': new_plate, 'trans':trans})
                count+=len(trans)
    compile_protocol = {key:[x for x in value if key != 'to' ] for key, value in itertools.groupby(sorted(protocol, key=itemgetter('to')), key=itemgetter('to'))}
    return [plate_merge(collection_name,k,i,v,{},None) for i, (k, v) in enumerate(compile_protocol.items())]
        
        
                        
compiled = compile_parts('FG_JCVISYN3',['9d0aeaa7-6ed5-4110-9759-4c6a0340acef','9198f7b6-3edd-4f7f-a52d-6c8ef0ac2044','4f425ab9-dc81-4e17-a9f6-52f10fcc000f','cbcfd9ae-7348-4448-9191-9c242e915f23','c8bc192b-557e-45b5-8831-a8292c8b6176'], [r])


[('dcf981a7-683c-4bad-acfe-8ed35217ec36', ['E7'])] [('7855f827-295a-4e8f-906d-788d38a3c5ac', ['A1'])]


In [12]:
plates = requests.get('https://api.freegenes.org/plates/').json()
plate_list = []
for plate in plates:
    if plate['plate_name'][-1] != 'a':
        plate_list.append(requests.get('https://api.freegenes.org/plates/recurse/{}'.format(plate['uuid'])).json())
parts = [x['uuid'] for x in requests.get('https://api.freegenes.org/collections/recurse/78c1a296-69ff-4e42-a2a8-9915bc84bd71').json()['parts']]


In [29]:
compiled = compile_parts('FG_JCVISYN3',parts,plate_list)

new
new
[('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['F7']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['H7']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['C9']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['F9']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['B10']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['E10']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['A11']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['C11']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['G11']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['C12']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['E12', 'F12']), ('a5b63ac0-bdcf-4dc2-91aa-c9e5cda6086e', ['H12']), ('66c4c1bc-6259-4036-a510-3ecdbeb40cf2', ['B1']), ('66c4c1bc-6259-4036-a510-3ecdbeb40cf2', ['D1']), ('66c4c1bc-6259-4036-a510-3ecdbeb40cf2', ['G1', 'H1']), ('66c4c1bc-6259-4036-a510-3ecdbeb40cf2', ['A2', 'B2']), ('66c4c1bc-6259-4036-a510-3ecdbeb40cf2', ['E2', 'F2', 'G2']), ('66c4c1bc-6259-4036-a510-3ecdbeb40cf2', ['C3', 'D3']), ('66c4c1bc-6259-4036-a510-3ecdbeb40cf2', ['G3'

In [30]:
with open('/home/koeng/Downloads/jcvisyn3_distro_0.json', 'w') as outfile:  
    json.dump(compiled[0], outfile)

In [10]:
def transfer_wells(from_plate:dict,to_plate:str,from_address:str,to_address:str,from_vol:int,total_to_vol:int):
    obj = {'wells': {'post': [], 'put': []}, 'samples': {'post': [], 'put': []}}
    for well in from_plate['wells']:
        if well['address'] == from_address:
            obj['wells']['put'].append({'uuid': well['uuid'],'volume': well['volume']-from_vol})
            obj['wells']['post'].append({'organism': well['organism'], 'plate_uuid': to_plate, 'media': 'lb', 'volume': total_to_vol, 'well_type': 'culture'})
            obj['samples']['post'].append({''})
            
    

In [19]:
from jsonschema import validate
import json
with open('/home/koeng/notebooks/validation/protocolSchemaV3.json') as json_file:  
    schema = json.load(json_file)

validate(instance=compiled[0], schema=schema)

In [27]:
compiled[0]

{'protocol-schema': '1.0.0',
 'metadata': {'protocolName': 'Plate merge to create FG_JCVISYN3-distro-0',
  'author': 'Keoni Gandall',
  'datetime': '2019-05-21 10:42:56.768090'},
 'default-values': {'aspirate-flow-rate': {'p10_single': 5,
   'p10_multi': 5,
   'p50_single': 25,
   'p50_multi': 25,
   'p300_single': 150,
   'p300_multi': 150,
   'p1000_single': 500},
  'dispense-flow-rate': {'p10_single': 10,
   'p10_multi': 10,
   'p50_single': 50,
   'p50_multi': 50,
   'p300_single': 300,
   'p300_multi': 300,
   'p1000_single': 1000},
  'aspirate-mm-from-bottom': 1,
  'dispense-mm-from-bottom': 0.5,
  'touch-tip-mm-from-top': -1},
 'designer-application': {'application-name': 'opentrons_json',
  'application-version': '1.0.0',
  'data': {}},
 'robot': {'model': 'OT-2 Standard'},
 'pipettes': {'1e504449-a1cf-44dd-8d09-ee266e5e5207': {'type': 'pipette',
   'mount': 'left',
   'name': 'p50_multi',
   'model': 'p50_multi_v1.3'}},
 'labware': {'11_tip_box': {'slot': '11',
   'model': 'op